# SQL Joins

### Import packages 
sqlite3
pandas
   
### Connect to databases cats.db, dogs.db and owners.db using sqlite3 (use for example conn = sqlite3.connect('cats.db') and then c = conn.cursor())

Let's try and work through some exercises to recap and retain the foundational knowledge of the language that our beloved databases use.

In the suburbs of Capitol Hill, there is a new pet daycare starting up that distributes pets among 4 houses.
Down below we have tables representing cats and dogs in the particular houses, and corresponding owners. 

Cats Table:

Name  			 | Breed 			| House_Number| Owner_ID|       
------------- | ------------- | ------------|---------
Bell  			| Siamese			|		1	    | 4
Jackson  		| Balinese 		|       2     |3
Precious  	| Himalayan  		|         3    | 4
Rocky			| Egyptian Mau 	| 4 		    | 2
Samson			| Javanese		|	4		| 1

Dogs Table:

Name  			 | Breed 	| House_Number	|Owner_ID|
-------------| ------------- | ------------|-----   
Rex  			| Chihuahua  		|		1	    |3
Clifford  	| German Shepherd|     2        |2
Lucky  		| Daschund  		|     3        |1
Bobo			| Shih-Tzu		|    2		   | 4
Buddy			| Golden Retriever | 4		   | 2
Leo				| English Bulldog | 1 | 2

Owners Table:

Name  		| Phone	| Address	|ID| 
-------------| ------------- | ------------|-----
Josh Daniell  | 313-287-9573  |1440 G St. |1
Alison Peebles Madigan| 214-709-8190| None |2
Avi Flombaum	| 469-878-0125  |  1776 New York Ave| 3
Justin Bieber | 410-381-0987 | None | 4


Write the appropriate SQL queries to satisfy the following prompts:

A. Write a SQL query that can grab the names of all pets owned by the founder of Flatiron School, Avi Flombaum

B. Which House_Number contains the most number of pets in it and how many?

C. The daycare realized that House 4 needs the owners of its pets to update their addresses. Grab the names and phone numbers of owners with pets in house 4 that do not currently have an address.



In [21]:
import sqlite3
import pandas as pd

conn_cats = sqlite3.connect('cats.db')
cur_cats = conn_cats.cursor()

conn_dogs = sqlite3.connect('dogs.db')
cur_dogs = conn_dogs.cursor()

conn_owners = sqlite3.connect('owners.db')
cur_owners = conn_owners.cursor()

In [50]:
cur_cats.execute("""SELECT * FROM cats LIMIT 5""")
df_cats = pd.DataFrame(cur_cats.fetchall())
df_cats.columns=[x[0] for x in cur_cats.description]
df_cats['type'] = 'cat'
df_cats

,Name,Breed,House_Number,Owner_ID,type
0,Bell,Siamese,1,4,cat
1,Jackson,Balinese,2,3,cat
2,Precious,Himalayan,3,4,cat
3,Rocky,Egyptian Mau,4,2,cat
4,Samson,Javanese,5,1,cat


In [51]:
cur_dogs.execute("""SELECT * FROM dogs""")
df_dogs = pd.DataFrame(cur_dogs.fetchall())
df_dogs.columns=[x[0] for x in cur_dogs.description]
df_dogs['type'] = 'dog'
df_dogs

,Name,Breed,House_Number,Owner_ID,type
0,Rex,Chihuahua,1,3,dog
1,Clifford,German Shepherd,2,2,dog
2,Lucky,Daschund,3,1,dog
3,Bobo,Shih-Tzu,2,4,dog
4,Buddy,Golden Retriever,4,2,dog
5,Leo,English Bulldog,1,2,dog


In [52]:
cur_owners.execute("""SELECT * FROM owners""")
df_owners = pd.DataFrame(cur_owners.fetchall())
df_owners.columns=[x[0] for x in cur_owners.description]
df_owners

,Name,Phone,Address,ID
0,Josh Daniell,3132879573,1440 G st.,1
1,Alison Peebles Madigan,2147098190,None,2
2,Avi Flombaum,4698780125,1776 New York Ave,3
3,Justin Bieber,4103810987,None,4


In [72]:
dfc = pd.merge(df_cats, df_owners, left_on='Owner_ID',right_on='ID',how='left')
dfd = pd.merge(df_dogs, df_owners, left_on='Owner_ID',right_on='ID',how='left')

df_all = pd.concat([dfc,dfd])
df_all

,Name_x,Breed,House_Number,Owner_ID,type,Name_y,Phone,Address,ID
0,Bell,Siamese,1,4,cat,Justin Bieber,4103810987,None,4
1,Jackson,Balinese,2,3,cat,Avi Flombaum,4698780125,1776 New York Ave,3
2,Precious,Himalayan,3,4,cat,Justin Bieber,4103810987,None,4
3,Rocky,Egyptian Mau,4,2,cat,Alison Peebles Madigan,2147098190,None,2
4,Samson,Javanese,5,1,cat,Josh Daniell,3132879573,1440 G st.,1
0,Rex,Chihuahua,1,3,dog,Avi Flombaum,4698780125,1776 New York Ave,3
1,Clifford,German Shepherd,2,2,dog,Alison Peebles Madigan,2147098190,None,2
2,Lucky,Daschund,3,1,dog,Josh Daniell,3132879573,1440 G st.,1
3,Bobo,Shih-Tzu,2,4,dog,Justin Bieber,4103810987,None,4
4,Buddy,Golden Retriever,4,2,dog,Alison Peebles Madigan,2147098190,None,2


In [67]:
# A. Write a SQL query that can grab the names of 
# all pets owned by the founder of Flatiron School, Avi Flombaum

df_all.query("ID == '3'")['Name_x']

1    Jackson
0        Rex
Name: Name_x, dtype: object

In [95]:
# B. Which House_Number contains the most number of pets in it and how many?

df_all[['Name_x','House_Number']].groupby('House_Number').agg(['count'])



,Name_x
,count
House_Number,
1,3
2,3
3,2
4,2
5,1


In [71]:
# C. The daycare realized that House 4 needs the owners of its pets to update their addresses. 
# Grab the names and phone numbers of owners with pets in house 4 
# that do not currently have an address.

df_all.query("House_Number == '4'")[['Name_y','Phone']]


,Name_y,Phone
3,Alison Peebles Madigan,2147098190
4,Alison Peebles Madigan,2147098190
